Sistema mas eficiente

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv(r'C:\Users\Fernando\Desktop\Proyecto 1\Proyecto-Individual-1-ML\data procesada para funciones\data_ml')
df=df[['Id_Pelicula','title', 'coleccion', 'overview', 'genero_1']]
df=df.fillna('')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45464 entries, 0 to 45463
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id_Pelicula  45464 non-null  object
 1   title        45464 non-null  object
 2   coleccion    45464 non-null  object
 3   overview     45464 non-null  object
 4   genero_1     45464 non-null  object
dtypes: object(5)
memory usage: 1.7+ MB


In [3]:
df['combined_features'] = df['overview'] + ' ' + df['genero_1'] + ' ' + df['coleccion']
df['combined_features']

0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
45459                                              Family 
45460                                           Animation 
45461                                         Documentary 
45462                                           Adventure 
45463                                             Mystery 
Name: combined_features, Length: 45464, dtype: object

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
matriz_tfidf_1 = tfidf.fit_transform(df['combined_features'])

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(matriz_tfidf_1, matriz_tfidf_1)

In [6]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

def get_recomendacion(title, cosine_sim=cosine_sim):
    if title not in indices:
        return "El titulo ingresado no se encuentra en el dataset, por favor vuelva a intentar"
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:6]

    movie_indices = [i[0] for i in sim_scores]

    return df['title'].iloc[movie_indices]

In [7]:
print(get_recomendacion('Django Unchained'))

18454    One for the Money
37130       Band of Angels
28302        Bare Knuckles
32536        Moving Target
39110     Into Pitch Black
Name: title, dtype: object


In [8]:
df.drop(['coleccion','overview','genero_1'],axis=1,inplace=True)

In [9]:
np.save('cosine_sim.npy', cosine_sim)
df.to_pickle('data_funcion_recomendacion')